In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import StandardScaler

In [2]:
DATA_PATH = '../data/'
# Read the CSV file into a DataFrame
df = pd.read_csv(DATA_PATH + 'raw/filtered.tsv', delimiter='\t')
# Display the first few rows of the dataset
df.head()

,id,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,0,"If Alkar is flooding her with psychic waste, t...","if Alkar floods her with her mental waste, it ...",0.785171,0.010309,0.014195,0.981983
1,1,Now you're getting nasty.,you're becoming disgusting.,0.749687,0.071429,0.065473,0.999039
2,2,"Well, we could spare your life, for one.","well, we can spare your life.",0.919051,0.268293,0.213313,0.985068
3,3,"Ah! Monkey, you've got to snap out of it.","monkey, you have to wake up.",0.664333,0.309524,0.053362,0.994215
4,4,I've got orders to put her down.,I have orders to kill her.,0.726639,0.181818,0.009402,0.999348


Select the relevant columns for training the text de-toxification model, which are 'reference', 'translation', 'ref_tox', and 'trn_tox'.


In [4]:
# Select the relevant columns for training the de-toxification model
selected_columns = ['reference', 'translation', 'ref_tox', 'trn_tox']

# Create a new DataFrame with selected columns
selected_data = df[selected_columns]
selected_data.head()

,reference,translation,ref_tox,trn_tox
0,"If Alkar is flooding her with psychic waste, t...","if Alkar floods her with her mental waste, it ...",0.014195,0.981983
1,Now you're getting nasty.,you're becoming disgusting.,0.065473,0.999039
2,"Well, we could spare your life, for one.","well, we can spare your life.",0.213313,0.985068
3,"Ah! Monkey, you've got to snap out of it.","monkey, you have to wake up.",0.053362,0.994215
4,I've got orders to put her down.,I have orders to kill her.,0.009402,0.999348


In [5]:
# Preprocess the text data
nltk.download('punkt')  # Download the Punkt tokenizer models
selected_data['reference'] = selected_data['reference'].apply(lambda text: ' '.join(word_tokenize(text)))
selected_data['translation'] = selected_data['translation'].apply(lambda text: ' '.join(word_tokenize(text)))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rafai\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
C:\Users\rafai\AppData\Local\Temp\ipykernel_13980\1539150184.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['reference'] = selected_data['reference'].apply(lambda text: ' '.join(word_tokenize(text)))
C:\Users\rafai\AppData\Local\Temp\ipykernel_13980\1539150184.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['translation'] = se

In [6]:
# Standardize the toxicity scores
scaler = StandardScaler()
selected_data['ref_tox'] = scaler.fit_transform(selected_data['ref_tox'].values.reshape(-1, 1))
selected_data['trn_tox'] = scaler.fit_transform(selected_data['trn_tox'].values.reshape(-1, 1))


C:\Users\rafai\AppData\Local\Temp\ipykernel_13980\3232388152.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['ref_tox'] = scaler.fit_transform(selected_data['ref_tox'].values.reshape(-1, 1))
C:\Users\rafai\AppData\Local\Temp\ipykernel_13980\3232388152.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['trn_tox'] = scaler.fit_transform(selected_data['trn_tox'].values.reshape(-1, 1))


In [7]:
selected_data.head()

,reference,translation,ref_tox,trn_tox
0,"If Alkar is flooding her with psychic waste , ...","if Alkar floods her with her mental waste , it...",-1.152122,1.193047
1,Now you 're getting nasty .,you 're becoming disgusting .,-1.040057,1.230213
2,"Well , we could spare your life , for one .","well , we can spare your life .",-0.716958,1.199768
3,"Ah ! Monkey , you 've got to snap out of it .","monkey , you have to wake up .",-1.066524,1.219701
4,I 've got orders to put her down .,I have orders to kill her .,-1.162597,1.230887


Split the data into 80% for training and 20% for testing

In [10]:
# Split the data into training, validation, and test sets
# 80% for training and 20% for testing
train_data, test_data = train_test_split(selected_data, test_size=0.2, random_state=42)


In [11]:
# Save the split datasets to separate CSV files
train_data.to_csv(DATA_PATH + 'interim/train.csv', index=False)
test_data.to_csv(DATA_PATH + 'interim/test.csv', index=False)
